In [1]:
"""
Created on Thu Feb  4 06:20:12 2016

@author: monkey
"""
# % pylab inline 
# % matplotlib inline

import pandas as pd
import xgboost as xgb
import csv
import numpy as np
import pdb
from copy import deepcopy
import random
# import matplotlib.pyplot as plt
from sklearn.metrics import log_loss

from sklearn.cross_validation import cross_val_score
from xgboost import XGBClassifier
# from bayes_opt import BayesianOptimization

from sklearn import cross_validation
from sklearn import ensemble
from sklearn import metrics
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [2]:
print('Load data...')
train = pd.read_csv("train.csv")
target = train['target']

train_with_target = train.drop(['ID'], axis = 1)
train = train.drop(['ID','target'],axis=1)
test = pd.read_csv("test.csv")
ids = test['ID'].values
test = test.drop(['ID'],axis=1)

# add number of nans
train['number_NAN'] = train.isnull().sum(axis = 1)
test['number_NAN'] = test.isnull().sum(axis = 1)
train_with_target['number_NAN'] = train.isnull().sum(axis = 1)

train_len = train.shape[0]
test_len = test.shape[0]

train = train.fillna(-1)
test = test.fillna(-1)
train_with_target = train_with_target.fillna(-1)

Load data...


In [3]:
sum(train.columns == test.columns)

132

# filter all object features

In [4]:
object_features = []
all_features = []
for feat in train.columns:
    if feat != 'number_NAN':
        all_features.append(feat)
    if train[feat].dtype == 'object':
        object_features.append(feat)

In [5]:
object_features.extend(['v38', 'v62', 'v72', 'v129'])
print object_features

['v3', 'v22', 'v24', 'v30', 'v31', 'v47', 'v52', 'v56', 'v66', 'v71', 'v74', 'v75', 'v79', 'v91', 'v107', 'v110', 'v112', 'v113', 'v125', 'v38', 'v62', 'v72', 'v129']


In [6]:
len(all_features)

131

In [7]:
train.head(3)

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v123,v124,v125,v126,v127,v128,v129,v130,v131,number_NAN
0,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,...,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144,1
1,-1.000000,-1.000000,C,-1.000000,9.191265,-1.000000,-1.000000,2.301630,-1.000000,1.312910,...,-1.000000,0.598896,AF,-1.000000,-1.000000,1.957825,0,-1.000000,-1.000000,81
2,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,...,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472,2


# add frequencies for object features

In [8]:
train.columns

Index([u'v1', u'v2', u'v3', u'v4', u'v5', u'v6', u'v7', u'v8', u'v9', u'v10', u'v11', u'v12', u'v13', u'v14', u'v15', u'v16', u'v17', u'v18', u'v19', u'v20', u'v21', u'v22', u'v23', u'v24', u'v25', u'v26', u'v27', u'v28', u'v29', u'v30', u'v31', u'v32', u'v33', u'v34', u'v35', u'v36', u'v37', u'v38', u'v39', u'v40', u'v41', u'v42', u'v43', u'v44', u'v45', u'v46', u'v47', u'v48', u'v49', u'v50', u'v51', u'v52', u'v53', u'v54', u'v55', u'v56', u'v57', u'v58', u'v59', u'v60', u'v61', u'v62', u'v63', u'v64', u'v65', u'v66', u'v67', u'v68', u'v69', u'v70', u'v71', u'v72', u'v73', u'v74', u'v75', u'v76', u'v77', u'v78', u'v79', u'v80', u'v81', u'v82', u'v83', u'v84', u'v85', u'v86', u'v87', u'v88', u'v89', u'v90', u'v91', u'v92', u'v93', u'v94', u'v95', u'v96', u'v97', u'v98', u'v99', u'v100', ...], dtype='object')

In [9]:
max_categories = 100

In [10]:
# prepare data
train_dummies = pd.DataFrame()
test_dummies = pd.DataFrame()

In [11]:
prob = train_with_target.groupby(['target']).size()[1]/float(sum(train_with_target.groupby(['target']).size()))

In [12]:
prob

0.76119872989214576

In [13]:
for elt in all_features:
    vector = pd.concat([train[elt], test[elt]], axis = 0)
    print str(elt)

    if len(vector.unique()) < max_categories:
        train_dummies = pd.concat([train_dummies, pd.get_dummies(train[elt], prefix = elt, \
                                                                 dummy_na = True)], axis = 1).astype('int8')
        test_dummies = pd.concat([test_dummies, pd.get_dummies(test[elt], prefix = elt, \
                                                               dummy_na = True)], axis = 1).astype('int8')
        del train[elt], test[elt]
    else:
        typ = str(train[elt].dtype)[:3]
        if (typ == 'flo') or (typ == 'int'):
            train[elt] = np.log(2 + train[elt])
            test[elt] = np.log(2 + test[elt])
        else:
            if (typ == 'obj'):
                list2keep = vector.value_counts()[ : max_categories].index
                train[elt] = train[elt].apply(lambda x: x if x in list2keep else np.nan)
                test[elt] = test[elt].apply(lambda x: x if x in list2keep else np.nan)                
                train_dummies = pd.concat([train_dummies, pd.get_dummies(train[elt], prefix = elt, \
                                                                         dummy_na = True)], axis = 1).astype('int8')
                test_dummies = pd.concat([test_dummies, pd.get_dummies(test[elt], prefix = elt, \
                                                                       dummy_na = True)], axis = 1).astype('int8')
                
                agrg = pd.concat([train[elt], target], axis = 1)
                agrg = agrg.groupby(by = elt, axis = 0).agg(['sum','count']).target
                agrg['weight'] = agrg.apply(lambda x: .5 + .5 * x['sum']/x['count'] if\
                                            (x['sum'] > x['count'] - x['sum']) else .5 + .5 * \
                                            (x['sum'] - x['count'])/x['count'], axis = 1)
                agrg.reset_index(inplace = True)
                train[elt + '_weight'] = pd.merge(train, agrg, how = 'left', on = elt)['weight']
                test[elt + '_weight'] = pd.merge(test, agrg, how = 'left', on = elt)['weight']
                train[elt + '_weight'] = train[elt + '_weight'].fillna(prob)
                test[elt + '_weight'] = test[elt + '_weight'].fillna(prob)
                del train[elt], test[elt]
            else:
                print('error', typ)

v1
v2
v3
v4
v5
v6
v7
v8
v9
v10
v11
v12
v13
v14
v15
v16
v17
v18
v19
v20
v21
v22
v23
v24
v25
v26
v27
v28
v29
v30
v31
v32
v33
v34
v35
v36
v37
v38
v39
v40
v41
v42
v43
v44
v45
v46
v47
v48
v49
v50
v51
v52
v53
v54
v55
v56
v57
v58
v59
v60
v61
v62
v63
v64
v65
v66
v67
v68
v69
v70
v71
v72
v73
v74
v75
v76
v77
v78
v79
v80
v81
v82
v83
v84
v85
v86
v87
v88
v89
v90
v91
v92
v93
v94
v95
v96
v97
v98
v99
v100
v101
v102
v103
v104
v105
v106
v107
v108
v109
v110
v111
v112
v113
v114
v115
v116
v117
v118
v119
v120
v121
v122
v123
v124
v125
v126
v127
v128
v129
v130
v131


In [14]:
train.head(3)

,v1,v2,v4,v5,v6,v7,v8,v9,v10,v11,...,v123,v124,v126,v127,v128,v130,v131,number_NAN,v22_weight,v56_weight
0,1.204694,2.372808,1.77851,2.294076,1.525899,1.644205,0.699597,2.484907,0.917602,2.914203,...,1.383736,0.710866,1.336086,1.631927,1.392347,0.969401,1.580451,1,0.761199,0.930675
1,0.000000,0.000000,0.00000,2.415134,0.000000,0.000000,1.458994,0.000000,1.197827,0.000000,...,0.000000,0.955087,0.000000,0.000000,1.375695,0.000000,0.000000,81,0.761199,0.930675
2,1.079727,1.989254,1.85801,1.991451,1.788352,1.779783,0.702922,2.685577,1.017353,2.818762,...,1.499086,0.699851,1.328058,1.778707,1.137983,1.058872,1.155771,2,0.761199,0.930675


In [15]:
for elt in train.columns:
    if (elt[-2:] == 'na') & (elt != 'v2_na'):
        print str(elt)
        dist = metrics.pairwise_distances(train.v2_na.reshape(1, -1), train[elt].reshape(1, -1))
        if dist < 8:
            del train[elt], test[elt]
        else:
            print(elt, dist)

In [16]:
train = pd.concat([train, train_dummies, target], axis = 1)
test = pd.concat([test, test_dummies], axis = 1)

In [41]:
train.shape

(114321, 621)

In [42]:
for elt in list(set(train.columns) - set(test.columns)):
    del train[elt]
for elt in list(set(test.columns) - set(train.columns)):
    del test[elt]

In [43]:
# tree blend

In [44]:
n_folds = 5
skf = list(StratifiedKFold(target, n_folds))

In [45]:
skf

[(array([ 22575,  22576,  22580, ..., 114318, 114319, 114320]),
  array([    0,     1,     2, ..., 22961, 22962, 22964])),
 (array([     0,      1,      2, ..., 114318, 114319, 114320]),
  array([22575, 22576, 22580, ..., 45728, 45729, 45731])),
 (array([     0,      1,      2, ..., 114318, 114319, 114320]),
  array([45724, 45726, 45730, ..., 68871, 68875, 68882])),
 (array([     0,      1,      2, ..., 114318, 114319, 114320]),
  array([68503, 68504, 68505, ..., 91556, 91559, 91560])),
 (array([    0,     1,     2, ..., 91556, 91559, 91560]),
  array([ 91424,  91425,  91426, ..., 114318, 114319, 114320]))]

In [46]:
rnd = 57
xgboost_params_1 = { 
        "objective": "binary:logistic",
        "booster": "gbtree",
        "eval_metric": "auc",
        "eta": 0.01, # 0.06, #0.01,
        "min_child_weight": 1,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "max_depth": 12,
        'silent': 1,
        'lambda': 0.5,
        'gamma': 0.5
    }

xgboost_params_2 = { 
        "objective": "binary:logistic",
        "booster": "gbtree",
        "eval_metric": "auc",
        "eta": 0.01, # 0.06, #0.01,
        "min_child_weight": 1,
        "subsample": 0.5,
        "colsample_bytree": 0.5,
        "max_depth": 12,
        'silent': 1,
        'lambda': 0.5,
        'gamma': 0.5
    }

clfs=[
    'xgboost-1',
    'xgboost-2',
    ensemble.RandomForestClassifier(bootstrap=False, class_weight='auto', criterion='entropy',
            max_depth = None, max_features='sqrt', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators = 3000, n_jobs=-1,
            oob_score=False, random_state=rnd, verbose=0,
            warm_start=False),
    ensemble.ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
           max_depth = None, max_features='sqrt', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=1e-5, n_estimators = 3000, n_jobs=-1,
           oob_score=False, random_state=rnd, verbose=0, warm_start=False)
]

'''
clfs=[
    ensemble.ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
           max_depth = None, max_features='sqrt', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=1e-5, n_estimators = 1000, n_jobs=-1,
           oob_score=False, random_state=rnd, verbose=0, warm_start=False)
]
'''

"\nclfs=[\n    ensemble.ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',\n           max_depth = None, max_features='sqrt', max_leaf_nodes=None,\n           min_samples_leaf=1, min_samples_split=2,\n           min_weight_fraction_leaf=1e-5, n_estimators = 1000, n_jobs=-1,\n           oob_score=False, random_state=rnd, verbose=0, warm_start=False)\n]\n"

In [47]:
xgb_rounds = 1500

In [48]:
X, y, X_submission = train, target, test

In [49]:
test.shape

(114393, 621)

In [50]:
print "Creating train and test sets for blending."
    
dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_submission.shape[0], len(clfs)))

Creating train and test sets for blending.


In [ ]:
for j, clf in enumerate(clfs):
    print j, clf
    dataset_blend_test_j = np.zeros((X_submission.shape[0], len(skf)))
    for i, (train_idx, test_idx) in enumerate(skf):
        print "Fold: ", i
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_test = X.iloc[test_idx]
        y_test = y.iloc[test_idx]
        if clf == 'xgboost-1':
            dtrain = xgb.DMatrix(X_train, label = y_train)
            dcv = xgb.DMatrix(X_test, label = y_test)
            clf_1 = xgb.train(xgboost_params_1, dtrain, num_boost_round = xgb_rounds)
            y_submission = clf_1.predict(dcv, ntree_limit = clf_1.best_iteration)
            dataset_blend_train[test_idx, j] = y_submission
            dsub = xgb.DMatrix(X_submission)
            dataset_blend_test_j[:, i] = clf_1.predict(dsub, ntree_limit = clf_1.best_iteration)
        elif clf == 'xgboost-2':
            dtrain = xgb.DMatrix(X_train, label = y_train)
            dcv = xgb.DMatrix(X_test, label = y_test)
            clf_1 = xgb.train(xgboost_params_2, dtrain, num_boost_round = xgb_rounds)
            y_submission = clf_1.predict(dcv, ntree_limit = clf_1.best_iteration)
            dataset_blend_train[test_idx, j] = y_submission
            dsub = xgb.DMatrix(X_submission)
            dataset_blend_test_j[:, i] = clf_1.predict(dsub, ntree_limit = clf_1.best_iteration)
        else:
            clf.fit(X_train, y_train)
            y_submission = clf.predict_proba(X_test)[:,1]
            dataset_blend_train[test_idx, j] = y_submission
            dataset_blend_test_j[:, i] = clf.predict_proba(X_submission)[:,1]
        print "log-loss: " + str(metrics.log_loss(y_test, y_submission))
    dataset_blend_test[:,j] = dataset_blend_test_j.mean(1)

In [ ]:
print
print "Stacking..."
# clf = LogisticRegression()
clf = ensemble.GradientBoostingClassifier(init = None, learning_rate = 0.1, loss = 'deviance',
              max_depth = 12, max_features = None, max_leaf_nodes = None,
              min_samples_leaf = 1, min_samples_split = 3,
              min_weight_fraction_leaf = 0.0, n_estimators = 10,
              random_state = rnd, subsample = 0.8, verbose = 0,
              warm_start = False)
clf.fit(dataset_blend_train, y)
y_submission = clf.predict_proba(dataset_blend_test)[:, 1]

print "Linear stretch of predictions to [0,1]"
y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())

In [ ]:
len(y_submission)

In [ ]:
y_submission

In [ ]:
# submit
submission = pd.read_csv('sample_submission.csv')
submission.PredictedProb = y_submission
submission.to_csv('stacking.csv', index=False)